<a href="https://colab.research.google.com/github/HwangHanJae/recommender_system/blob/main/book_Recommender_Systems/Model_Base_NMF_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NMF(Non-negative Matrix Factorization)
NMF는 음수 미포함 행렬 분해(비음행렬 인수분해)라고 하여 말 그대로 음수를 포함하지 않는 행렬 $X$를 음수를 포함하지 않는 행렬 $W$와 $H$의 곱으로 분해하는 알고리즘입니다.

$$X = WH$$

NMF와 관련하여 참고한 자료는 다음과 같습니다.  
참고링크1 : https://angeloyeo.github.io/2020/10/15/NMF.html  
참고링크2 : https://github.com/ahmadvh/Non-Negative-Matrix-factorization---Implemented-in-python


여기서는 음수를 포함하지 않는 평점을 가진 행렬을 비음평점 행렬 혹은 비음행렬이라고 표현하겠습니다.

비음평점 행렬은 암시적 피드백 데이터세트에 일반적으로 사용됩니다.

암시적 피드백 데이터 세트의 예는 다음과 같습니다.
1. 사용자가 특정 아이템에 대하여 구매 하였을 때 
2. 웹에서 사용자가 특정 아이템에 대해서 클릭 하였을 때
3. 페이스북의 "좋아요" 버튼을 선택하였을 때

위의 예처럼 사용자의 표현은 음수가 아닌 임의의 숫자로 표현할 수 있고, 이러한 숫자는 아이템에 대해서 호감도를 명시하지만 비호감도를 나타내는 것은 아닙니다.

즉 암시적 피드백 데이터 세트는 신뢰를 나타내고 명시적 피드백의 경우는 선호를 나타냅니다.

우리의 목적은 $WH$가 최대한 $X$와 가깝게 만드는 것입니다.
최대한 $X$와 가깝다 라는 의미는 $WH$와 $X$의 차이가 0에 가깝다라는 의미이고 우리가 최소화 해야하는 값입니다.

즉 우리의 목적함수는 프로베니우스 노름(Frobenius norm) $||X-WH||^{2}_{F}$ 정의할 수 있습니다. 

$$min(||X-WH||^{2}_{F})$$

즉 프로베니우스 노름(놈)이 0에 가까워야 하는 것이며 거리(유클리드)가 0에 가깝다는 것입니다.

우리의 목적 함수를 정의하였고 경사하강법을 이용하여 $W$, $H$업데이트 할수 있습니다.

위의 [참고링크1](https://angeloyeo.github.io/2020/10/15/NMF.html)을 보면 $W$, $H$ 행렬을 업데이트하는 과정이 상세하게 잘 나와 있습니다.


결론만 가지고 오면

$$H \Leftarrow H \circ \frac{W^{T}X}{W^{T}WH}$$
$$W \Leftarrow W \circ \frac{XH^{T}}{WHH^{T}}$$  


여기서 $\circ$ 기호는 원소별 곱(Hadamard product)를 의미합니다.


$\epsilon \approx 10^{-9}$와 같은 작은 값으로 분모가 0이 되지 않게 하여 안정성을 높입니다. $U$, $V$가 업데이트 될때 모든 항목이 "동시에" 업데이트가 됩니다.  
그리고 초기화 단계에서는 (0, 1)의 임의의 값으로 초기화 됩니다.

$$H \Leftarrow H \circ \frac{W^{T}X}{W^{T}WH + \epsilon}$$
$$W \Leftarrow W \circ \frac{XH^{T}}{WHH^{T}+ \epsilon}$$  


In [228]:
import numpy as np

def random_initialization(A,rank):
    number_of_documents = A.shape[0]
    number_of_terms = A.shape[1]
    W = np.random.uniform(0,1,(number_of_documents,rank))
    H = np.random.uniform(0,1,(rank,number_of_terms))
    return W,H
                          
def mu_method(A,k,max_iter,init_mode='random'):
    
    if init_mode == 'random':
        W ,H = random_initialization(A,k)
    norms = []
    e = 1.0e-10
    for n in range(max_iter):
        # Update H
        W_TA = W.T@A
        W_TWH = W.T@W@H+e
        for i in range(np.size(H, 0)):
            for j in range(np.size(H, 1)):
                H[i, j] = H[i, j] * W_TA[i, j] / W_TWH[i, j]
        # Update W
        AH_T = A@H.T
        WHH_T =  W@H@H.T+ e

        for i in range(np.size(W, 0)):
            for j in range(np.size(W, 1)):
                W[i, j] = W[i, j] * AH_T[i, j] / WHH_T[i, j]

        norm = np.linalg.norm(A - W@H, 'fro')
        norms.append(norm)
    return W ,H ,norms 

In [225]:
R = np.array([[2, 2, 1, 2, 0, 0],
       [2, 3, 3, 3, 0, 0],
       [1, 1, 1, 1, 0, 0],
       [2, 2, 2, 3, 1, 1],
       [0, 0, 0, 1, 1, 1],
       [0, 0, 0, 2, 1, 2]])

In [248]:
class NMF_CF:
  def __init__(self, R, k, max_iters):
    self.R =R
    self.num_users, self.num_items = R.shape
    self.k = k
    self.max_iters =max_iters
  
  def fit(self):
    W = np.random.uniform(0, 1, size=(self.num_users, self.k))
    H = np.random.uniform(0, 1, size=(self.k, self.num_items))
    

    norms = []
    e = 1e-10
    for n in range(self.max_iters):
        # Update H
        W_TA = W.T@self.R 
        W_TWH = W.T@W@H+e
        for i in range(np.size(H, 0)):
            for j in range(np.size(H, 1)):
                H[i, j] = H[i, j] * W_TA[i, j] / W_TWH[i, j]
        # Update W
        AH_T = self.R@H.T
        WHH_T =  W@H@H.T+ e

        for i in range(np.size(W, 0)):
            for j in range(np.size(W, 1)):
                W[i, j] = W[i, j] * AH_T[i, j] / WHH_T[i, j]

        norm = np.linalg.norm(self.R - W@H, 'fro')
        norms.append(norm)
    return norms

In [250]:
nmf = NMF_CF(R, k=2, max_iters=100)
nmf.fit()

[3.766430298821842,
 3.529562559919303,
 3.423481619119008,
 3.345343936087621,
 3.2804398228387726,
 3.224479458351885,
 3.175708652466205,
 3.132009364467895,
 3.090699431203165,
 3.0491479136593673,
 3.0050945325387373,
 2.9566025410103327,
 2.901806911965148,
 2.838482942802815,
 2.76336111321954,
 2.6713006672571944,
 2.5553850009940904,
 2.4107108619224777,
 2.242623930590698,
 2.0688255439099987,
 1.9069900196181575,
 1.7653894796392235,
 1.6456460775038657,
 1.5469843638719045,
 1.4676922789436715,
 1.4053962136198397,
 1.3573122305926508,
 1.3206078296135917,
 1.2927128420906193,
 1.2714848360740358,
 1.2552399613495422,
 1.242703985251357,
 1.2329335191043849,
 1.2252372197002004,
 1.2191095220284416,
 1.2141795228697327,
 1.2101733283275953,
 1.2068869696049462,
 1.2041671206167368,
 1.2018973982581964,
 1.1999885999061932,
 1.1983717048438491,
 1.1969928184175131,
 1.1958094882724852,
 1.1947879962627272,
 1.1939013496688822,
 1.1931277778937923,
 1.1924495976974847,
 1.191